In [9]:
import pandas as pd
import numpy as np
import re

In [10]:
df = pd.read_csv('2019-09-30.csv')
df_2 = pd.read_csv('2019-09-29.csv')

In [11]:
df.shape

(115150, 129)

In [12]:
type(df.columns)

pandas.core.indexes.base.Index

In [4]:
def prepare(df):
    
    # Drop normalized columns
    columns = []
    for i in df.columns:
        columns.append(i)
    columns = str(columns)
    normalized_columns = re.findall(r'(smart_\d+_normalized)', columns)
    df.drop(columns=normalized_columns, inplace=True)
    
    # Convert capacity column from bytes to gigabytes
    df['capacity_bytes'] = round((df['capacity_bytes']/ 1_000_000_000),1)

    
    # Convert power hours to days
    df['smart_9_raw'] = round((df['smart_9_raw']/ 24),1)

    # Rename columns appropriately
    df.rename(columns={'capacity_bytes':'capacity_gigabytes','smart_9_raw':'drive_age_in_days'}, inplace = True)
    
    return df

In [5]:
prepare(df)

,date,serial_number,model,capacity_gigabytes,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,...,smart_233_raw,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw,smart_255_raw
0,2019-09-30,Z305B2QN,ST4000DM000,4000.8,0,88847920.0,NaN,0.0,13.0,0.0,...,NaN,NaN,32985.0,5.061792e+10,1.621052e+11,NaN,NaN,NaN,NaN,NaN
1,2019-09-30,ZJV0XJQ4,ST12000NM0007,12000.1,0,53191496.0,NaN,0.0,3.0,0.0,...,NaN,NaN,9502.0,5.079863e+10,1.267450e+11,NaN,NaN,NaN,NaN,NaN
2,2019-09-30,ZJV0XJQ3,ST12000NM0007,12000.1,0,107432696.0,NaN,0.0,1.0,0.0,...,NaN,NaN,6961.0,4.913090e+10,4.643564e+10,NaN,NaN,NaN,NaN,NaN
3,2019-09-30,ZJV0XJQ0,ST12000NM0007,12000.1,0,198398656.0,NaN,0.0,6.0,0.0,...,NaN,NaN,10653.0,5.336751e+10,9.404781e+10,NaN,NaN,NaN,NaN,NaN
4,2019-09-30,PL1331LAHG1S4H,HGST HMS5C4040ALE640,4000.8,0,0.0,103.0,436.0,9.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-09-30,ZA16NQJR,ST8000NM0055,8001.6,0,54212352.0,NaN,0.0,7.0,0.0,...,NaN,NaN,21158.0,5.856080e+10,1.378226e+11,NaN,NaN,NaN,NaN,NaN
6,2019-09-30,ZJV02XWG,ST12000NM0007,12000.1,0,68148000.0,NaN,0.0,8.0,0.0,...,NaN,NaN,12006.0,5.196929e+10,9.946620e+10,NaN,NaN,NaN,NaN,NaN
7,2019-09-30,ZJV1CSVX,ST12000NM0007,12000.1,0,70300656.0,NaN,0.0,19.0,0.0,...,NaN,NaN,10412.0,5.410184e+10,1.393280e+11,NaN,NaN,NaN,NaN,NaN
8,2019-09-30,ZJV02XWA,ST12000NM0007,12000.1,0,174674360.0,NaN,0.0,7.0,0.0,...,NaN,NaN,12114.0,6.001205e+10,1.366960e+11,NaN,NaN,NaN,NaN,NaN
9,2019-09-30,ZA18CEBS,ST8000NM0055,8001.6,0,200448504.0,NaN,0.0,2.0,0.0,...,NaN,NaN,18143.0,5.159643e+10,1.324788e+11,NaN,NaN,NaN,NaN,NaN
